In [1]:
!git clone https://github.com/saisharank/Left-and-Right
!git clone https://github.com/tensorflow/models.git
!git clone https://github.com/kwotsin/create_tfrecords

Cloning into 'Left-and-Right'...
remote: Enumerating objects: 532, done.
remote: Counting objects: 100% (532/532), done.
remote: Compressing objects: 100% (532/532), done.
remote: Total 532 (delta 0), reused 532 (delta 0), pack-reused 0
Receiving objects: 100% (532/532), 27.17 MiB | 22.11 MiB/s, done.
Cloning into 'models'...
remote: Enumerating objects: 32650, done.
remote: Total 32650 (delta 0), reused 0 (delta 0), pack-reused 32650
Receiving objects: 100% (32650/32650), 511.53 MiB | 37.90 MiB/s, done.
Resolving deltas: 100% (20783/20783), done.
Checking out files: 100% (3174/3174), done.
Cloning into 'create_tfrecords'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 65 (delta 0), reused 0 (delta 0), pack-reused 62
Unpacking objects: 100% (65/65), done.


In [2]:
%cd /content/models/research/slim/datasets
!touch hp_plants.py

/content/models/research/slim/datasets


In [0]:
%cd /content/

/content


In [4]:
!python3 /content/create_tfrecords/create_tfrecord.py --dataset_dir=/content/Left-and-Right --tfrecord_filename=leftright --validation_size=0.2

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




2019-11-19 11:25:30.815505: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-11-19 11:25:30.864535: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-11-19 11:25:30.865181: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciBusID: 0000:00:04.0
2019-11-19 11:25:30.879134: I tensorflow/stream_executor/

In [0]:
train_data_dir = '/content/sides/training'
validation_data_dir = '/content/sides/testing'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 300
batch_size = 32

In [0]:
base_model=InceptionV3(input_shape=(224,224,3),weights='imagenet',include_top=False) #imports the inceptionResNetV2 model and discards the last 1000 neuron layer.
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu',kernel_regularizer=regularizers.l2(0.001))(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu',kernel_regularizer=regularizers.l2(0.001))(x) #dense layer 2
x=Dropout(0.2)(x)
x=Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.001))(x) #dense layer 3
preds=Dense(2,activation='softmax',kernel_regularizer=regularizers.l2(0.001))(x) #final layer with softmax activation

In [0]:
model=Model(inputs=base_model.input,outputs=preds)
#specify the inputs
#specify the outputs
#now a model has been created based on our architecture

In [0]:
for i,layer in enumerate(model.layers):
  print(i,layer.name)

In [0]:
#if we want to set the first 20 layers of the network to be non-trainable
for layer in model.layers[:20]:
  layer.trainable=False
for layer in model.layers[20:]:
  layer.trainable=True

In [0]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(rescale=1. / 255)
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [0]:
train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=batch_size, class_mode='categorical')

In [0]:
validation_generator = test_datagen.flow_from_directory(validation_data_dir, target_size=(img_width, img_height),
                                                        batch_size=batch_size, class_mode='categorical')

In [0]:
from keras.callbacks import ModelCheckpoint
savepath = os.path.join( ""+ 'e1-{epoch:03d}-vl-{val_loss:.3f}-va-{val_acc:.3f}.h5' )
checkpointer = ModelCheckpoint(filepath=savepath)

In [0]:
model.summary()

In [0]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

#step_size_train=train_generator.n//train_generator.batch_size
step_size_train=4000 // batch_size

history = model.fit_generator(generator=train_generator, steps_per_epoch=step_size_train, epochs = epochs, validation_steps=800 // batch_size,
                              validation_data=validation_generator,callbacks=[checkpointer])

In [0]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Retrieve a list of accuracy results on training and validation data
# sets for each training epoch
acc = history.history['acc']
val_acc = history.history['val_acc']

# Retrieve a list of list results on training and validation data
# sets for each training epoch
loss = history.history['loss']
val_loss = history.history['val_loss']

# Get number of epochs
epochs = range(len(acc))

# Plot training and validation accuracy per epoch
plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Training and validation accuracy')

plt.figure()

# Plot training and validation loss per epoch
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Training and validation loss')

In [0]:
 import os
import os.path as osp
import argparse

import tensorflow as tf

from keras.models import load_model
from keras import backend as K




def convertGraph( modelPath, outdir, numoutputs, prefix, name):
    '''
    Converts an HD5F file to a .pb file for use with Tensorflow.
    Args:
        modelPath (str): path to the .h5 file
           outdir (str): path to the output directory
       numoutputs (int):   
           prefix (str): the prefix of the output aliasing
             name (str):
    Returns:
        None
    '''
    
    #NOTE: If using Python > 3.2, this could be replaced with os.makedirs( name, exist_ok=True )
    if not os.path.isdir(outdir):
        os.mkdir(outdir)

    K.set_learning_phase(0)

    net_model = load_model(modelPath)

    # Alias the outputs in the model - this sometimes makes them easier to access in TF
    pred = [None]*numoutputs
    pred_node_names = [None]*numoutputs
    for i in range(numoutputs):
        pred_node_names[i] = prefix+'_'+str(i)
        pred[i] = tf.identity(net_model.output[i], name=pred_node_names[i])
    print('Output nodes names are: ', pred_node_names)

    sess = K.get_session()
    
    # Write the graph in human readable
    f = 'graph_def_for_reference.pb.ascii'
    tf.train.write_graph(sess.graph.as_graph_def(), outdir, f, as_text=True)
    print('Saved the graph definition in ascii format at: ', osp.join(outdir, f))

    # Write the graph in binary .pb file
    from tensorflow.python.framework import graph_util
    from tensorflow.python.framework import graph_io
    constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), pred_node_names)
    graph_io.write_graph(constant_graph, outdir, name, as_text=False)
    print('Saved the constant graph (ready for inference) at: ', osp.join(outdir, name))

convertGraph( "/content/e1-015-vl-0.497-va-0.885.h5", "/content/", 1,'output', 'output_graph.pb' )


In [0]:

import tensorflow as tf
from tensorflow import lite

print( tf.VERSION )
#converter = lite.TFLiteConverter.from_keras_model_file( '/content/sides/e1-001-vl-2.491-va-0.472.h5' ) 
input_arrays = ["input_1_1"]
output_arrays = ["output_0"]

converter = lite.TFLiteConverter.from_keras_model_file( '/content/e1-015-vl-0.497-va-0.885.h5' ) 
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)
interpreter = lite.Interpreter(model_path="/content/converted_model.tflite")
print(interpreter.get_output_details()[0]['shape'])

In [0]:
import tensorflow as tf
from tensorflow.python.platform import gfile

f = gfile.FastGFile("/content/output_graph.pb", 'rb')
graph_def = tf.GraphDef()
# Parses a serialized binary message into the current message.
graph_def.ParseFromString(f.read())
f.close()

#sess.graph.as_default()
# Import a serialized TensorFlow `GraphDef` protocol buffer
# and place into the current default `Graph`.
tf.import_graph_def(graph_def)
print('Check out the input placeholders:')
nodes = [n.name + ' => ' +  n.op for n in graph_def.node if n.op in ('Placeholder')]
for node in nodes:
    print(node)

In [0]:
 model.save_weights('first_try.h5')